In [ ]:
## For this process, we will primarily use PyGrib



---
### Before running through this notebook, make sure to complete these notebooks
- [Atmos Variables (3-Hourly Data)](Atmos_Deltas.ipynb)
- [Soil Variables (Monthly Data)](Soil_Deltas.ipynb)

---


#### Running these notebooks will create graphics to visualize the climate change deltas, in addition to netCDF files that contain the data
---



### **Import libraries and setup file paths**


#### Now that teverything is setup, let's import the neccessary libraries

In [1]:
import pygrib
import ncepgrib2
import xarray as xr
import iris
import eccodes
import cfgrib
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.units import units
import numpy as np
from scipy.ndimage import gaussian_filter
import xarray as xr
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
import os
%pylab inline
import glob, os
from datetime import datetime

Populating the interactive namespace from numpy and matplotlib


## Run in parallel

In [ ]:
from dask.distributed import Client, progress
client = Client(dashboard_address=':9999', memory_limit='10GB', n_workers=20, threads_per_worker=1)  # set up local cluster on your laptop
client

In [ ]:
client.close()

---
### **Write to File**


### This function loops through grib file and edit values with climate change delta

In [7]:
def add_delta_rh(temp_ds, delta_ds, rh_vals, level='pressure', plev=None, pres_ds=None):
    """

    Add relative humidity delta given temperature, specific humidity, and original relative humidity

    Parameters
    ===========
    temp_ds = temperature dataset (3-dimensional) with pressure levels in isobariInhPa

    spec_hum_ds = specific humidity (3-dimensional) with pressure levels expressed as level

    rh_vals = relative humidity (2-dimensional)

    plev = pressure level

    ==========
    Returns
    ==========
    modified_rh = modified relative humidity values

    """
    
    if level == 'pressure':
        
        # Assign units and bring in values of temperature, specific humidity, and pressure level
        temp = temp_ds['t'].sel(isobaricInhPa=plev).values * units.kelvin
        spec_hum_delta = delta_ds['hus'].sel(plev=plev).values * units('kg/kg')
        plev = plev * units.hPa
        
        # Calculate new temperature
        temp_new = temp + (delta_ds['ta'].sel(plev=plev).values * units.kelvin)
        
    if level == 'surface':
        temp = temp_ds['t2m'].values * units.kelvin
        plev = pres_ds['sp'].values * units.Pa
        spec_hum_delta = delta_ds['huss'].values * units('kg/kg')
        
        # Calculate new temperature
        temp_new = temp + (delta_ds['tas'].values * units.kelvin)
    

    # Calcualte mixing ratio using relative humidity and temperature
    w = mcalc.mixing_ratio_from_relative_humidity(rh_vals * units.percent, temp, plev)
    
    # Convert from specific humidity to mixing ratio
    spec_hum_delta = mcalc.mixing_ratio_from_specific_humidity(spec_hum_delta)
    
    # Caclulate new mixing rato
    w_new = w + spec_hum_delta
    
    # Calculate new relative humidity (with new temperature value)
    rh_new = mpcalc.relative_humidity_from_mixing_ratio(w_new, temp_new, plev)

    # Return relative humidity values, setting all values greater than 100% to 100%
    vals_not_over_100 = np.where(rh_new < 100.*units.percent, np.round(rh_new.to('percent'), 0), 100)
    
    # Make sure RH values less than 0% are set to 0
    vals_not_negative = np.where(vals_not_over_100 > 0, vals_not_over_100, 0)
    
    # Return the refined numpy array
    return np.around(vals_not_negative, decimals=4)

In [8]:
def modify_grib_file(model, nam_filepath, namfile, output_file_path, setting, time_frequency='hourly', plot_3d=False, plot_sfc=False, plot_soil=False):
    """
    Uses climate change deltas to modify NAM analysis files
    
    Input:

    Model: String, Type of model (ex. MIROC, GFDL, or NCAR)
    
    namfile: String, Name of nam file (full filepath)
    
    output_file_path: string, file path where the output will be saved
    
    setting: String, Variables to modify with climate change deltas.
             Options: Atmos, Surface, or Soil. Use All to edit all three
    
    plot_3d: Boolean, creates 500 hPa (Height and Winds) and 700 hPa (Height and RH)
    
    plot_2d: Boolean, creates surface map (temperature and MSLP)
    
    Output:
    
    Modified name files: saved in output_file directory
    
    Figures: Creates maps
    """
    print(time_frequency)
    
    ds_2d = xr.open_dataset('../preliminary_netcdf/'+model+'_2d_vars_3hr.nc')
    ds_3d = xr.open_dataset('../preliminary_netcdf/'+model+'_3d_vars_3hr.nc')
    ds_soil = xr.open_dataset('../preliminary_netcdf/'+model+'_soil_vars.nc')
    
    # Read in the temperature dataset to be used for calculating the RH delta
    temp_ds = xr.open_dataset(nam_filepath + namfile, backend_kwargs={'filter_by_keys':{'typeOfLevel':'isobaricInhPa', 'cfName':'air_temperature'}},
                              engine='cfgrib').metpy.parse_cf()
    
    # Read in the temperature at the surface to be used for the RH surface delta
    temp_surface_ds = xr.open_dataset(nam_filepath + namfile, backend_kwargs={'filter_by_keys':{'typeOfLevel':'heightAboveGround', 'cfName':'air_temperature'}},
                              engine='cfgrib').metpy.parse_cf()
    
    # Read in the pressure dataset to be used for the RH surface delta
    pres_ds = xr.open_dataset(nam_filepath + namfile, backend_kwargs={'filter_by_keys':{'cfName':'surface_air_pressure'}},
                              engine='cfgrib').metpy.parse_cf()
    
    # Set Lats and Lons for plotting
    lats = ds_2d.lat
    lons = ds_2d.lon
    
    # Set the file to namfile
    file = namfile
    
    # Set the time
    dt = datetime.strptime(namfile[11:-8], '%Y%m%d_%H%M')
    
    # Test this out
    atmos_vars = ['ta', 'ua', 'va', 'hus']
    
    if setting == 'All':
        setting = 'Atmos, Surface, Soil'
    
    if 'Atmos' not in setting:
        atmos_vars = ['ta', 'va', 'ua', 'hus']
        for var in atmos_vars:
            ds_3d[var].values = ds_3d[var].values * 0
    
    if 'Surface' not in setting:
        
        surface_vars = ['psl', 'ps', 'tas', 'uas', 'vas', 'ts', 'huss']
    
        for var in surface_vars:
            try:
                ds_2d[var].values = ds_2d[var].values * 0
            
            except KeyError:
                #ds_2d[var].values = ds_2d['ps'].values * 0
                print('Missing variable: ', var)
            
    if 'Soil' not in setting:
        soil_vars = ['Soil Temperature', 'Soil Moisture']
        
        # Loop through the different variables and set = 0
        for var in soil_vars:
            ds_soil[var].values = (ds_soil[var].values * 0) + 1
    
    # Read in the data using pygrib
    grbs = pygrib.open(nam_filepath + file)
    
    lats = ds_2d.lat
    lons = ds_2d.lon
    
    if setting == 'Atmos, Surface, Soil':
        grbout = open(output_file_path +'/All/'+ model +'_' + namfile,'wb')
    else:
        grbout = open(output_file_path +'/'+setting+'/'+ model +'_' + namfile,'wb')
    
    hour = int(grbs[1].validityTime/100)
    
    # Check to see if hourly subset is needed
    if time_frequency == 'hourly':
        ds_3d_subset = ds_3d.sel(hour=hour)
        ds_2d_subset = ds_2d.sel(hour=hour)
    
    # Check to see if monthly subset is needed - compute if neccessary
    elif time_frequency == 'monthly':
        ds_3d_subset = ds_3d.mean('hour')
        ds_2d_subset = ds_2d.mean('hour')
    
    for grb in grbs:
        
        initial= grb.values
    
        level = grb.level
    
        # Isobaric surface variables 3D
        if grb.typeOfLevel == 'isobaricInhPa':
    
            # Modify temperature fields
            if grb.cfName == 'air_temperature':
                initial = grb['values']
                grb['values'] = grb.values + ds_3d_subset['ta'].sel(plev = level).values
                grbout.write(grb.tostring())
                test_zero = grb.values - initial
                #plot_var_atmos(grb, 'Temperature at ', model)
                
        
            # 3-D V-component of wind
            elif grb.cfName == 'northward_wind':
                initial = grb['values']
                grb['values'] = grb.values + ds_3d_subset['va'].sel(plev = level).values
                grbout.write(grb.tostring())
                test_zero = grb.values - initial
                #plot_var_atmos(grb, 'V-Wind at', model)
                
                if grb.level == 500:
                    vwnd_500 = gaussian_filter(np.array(grb['values']), sigma=3.0) * units('m/s')
        
            # 3-D U-component of wind
            elif grb.cfName == 'eastward_wind':
                initial = grb.values
                grb['values'] = grb.values + ds_3d_subset['ua'].sel(plev = level).values
                grbout.write(grb.tostring())
                #plot_var_atmos(grb, 'U-Wind at ', model)
                test_zero = grb.values - initial
                
                if grb.level == 500:
                    uwnd_500 = gaussian_filter(np.array(grb['values']), sigma=3.0) * units('m/s')
        
            # 3-D Relative
            elif grb.name == 'Relative humidity':
                initial = grb.values
                grb['values'] = add_delta_rh(temp_ds, ds_3d_subset, grb.values, plev=grb.level)
                
                grbout.write(grb.tostring())
                
                
                test_zero = grb.values - initial
                
                #plot_var_atmos(grb, 'RH at', model)
            
            # 3-D Geopotential Height
            #elif grb.cfName == 'geopotential_height':
                
                # Check to see if atmos deltas are set to 0
            #    if np.mean(ds_3d_subset['ta'].values) == 0:
                    
            #        print('Using default geopotential height')
            #        grbout.write(grb.tostring())
                
                # If delta is not 0, do not pass through geopotential height
            #    else:
            #        None

            else:
                None

        # Variables near surface - 2D
        elif grb.typeOfLevel == 'heightAboveGround':
        
            # 2m Temperature
            if grb.name == '2 metre temperature':

                grb['values'] = np.ma.array(grb.values + ds_2d_subset['tas'].values, mask=False)
                
                grbout.write(grb.tostring())
                
                tmpk = gaussian_filter(np.array(grb['values']), sigma=3.0) * units.kelvin
                
            elif grb.name == 'Pressure':
                
                grb['values'] = np.ma.array(grb.values + ds_2d_subset['ps'].values, mask=False)
                
                grbout.write(grb.tostring())
                
            elif grb.name == 'Temperature':
                

                # Try to use skin temperature - if an error comes up, use tas (2m temperature)
                try:
                    grb['values'] = np.ma.array(grb.values + ds_2d_subset['ts'].values, mask=False)
                    
                except:
                    grb['values'] = np.ma.array(grb.values + ds_2d_subset['tas'].values, mask=False)
                
                grbout.write(grb.tostring())
        
            # Relative humidty at 2m above surface
            elif grb.name == 'Relative humidity':

                grb['values'] = np.ma.array(add_delta_rh(temp_surface_ds, ds_2d_subset, grb.values, level='surface', pres_ds=pres_ds), mask=False)
                
                grbout.write(grb.tostring())

            
            # U-wind at 10 meters above ground
            elif grb.name == '10 metre U wind component':
                try:
    
                    # Modify the surface u-component of the wind
                    grb['values'] = np.ma.array(grb.values + ds_2d_subset['uas'].values, mask=False)

                    grbout.write(grb.tostring())


                except KeyError:
                    grbout.write(grb.tostring())

            # V-wind at 10 meters above ground
            elif grb.name == '10 metre V wind component':
                try:

                    # Modify the surface wind component
                    grb['values'] = np.ma.array(grb.values + ds_2d_subset['vas'].values, mask=False)
                    
                    grbout.write(grb.tostring())

                    
                except KeyError:
                    # If the surface wind component is missing (as in NCAR), use the surface winds from the NAM
                    grbout.write(grb.tostring())
                    
            elif grb.name == 'Specific humidity':
                
                # Modify the specific humidity near the surface
                grb['values'] = np.ma.array(grb.values + ds_2d_subset['huss'].values, mask=False)
                grbout.write(grb.tostring())
            
            else:
                None
    
        elif grb.typeOfLevel == 'surface':
            
            if grb.cfName == 'surface_air_pressure':

                grb['values'] = np.ma.array(grb.values + ds_2d_subset['ps'].values, mask=False)
                
                grbout.write(grb.tostring())


            # Surface temperature
            elif grb.cfName == 'air_temperature':
                try:

                    grb['values'] = np.ma.array(grb.values + ds_2d_subset['ts'].values, mask=False)
                    
                    grbout.write(grb.tostring())
                
                except:

                    print('Using TAS for TS')

                    grb['values'] = np.ma.array(grb.values + ds_2d_subset['tas'].values, mask=False)
                    
                    grbout.write(grb.tostring())
            
            # Use the original land-sea mask
            elif grb.name == 'Land-sea mask':
                grbout.write(grb.tostring())
                
            # Use the original orography
            elif grb.name == 'Orography':
                grbout.write(grb.tostring())
    
            # Sea ice
            elif grb.indicatorOfParameter == 91:
                grbout.write(grb.tostring())
    
            # Water equivalent snow depth
            elif grb.indicatorOfParameter == 65:
                grbout.write(grb.tostring())
        
            # Snow depth
            elif grb.indicatorOfParameter == 66:
                print(grb)
        
            # Plant canopy surface water
            elif grb.indicatorOfParameter == 223:
                print(grb)
            
            else:
                None
                
        # Modify soil temperature values
        elif grb.name == 'Soil Temperature':

            grb['values'] = np.ma.array(grb.values * (ds_soil['Soil Temperature'].sel(depth = grb.level).values), mask=False)
            
            grbout.write(grb.tostring())
            
            soil_t = grb['values'] * units.kelvin
        
        # Mean sea-level pressure
        elif grb.indicatorOfParameter == 130:

            grb['values'] = np.ma.array(grb.values + ds_2d_subset['psl'].values)
            
            grbout.write(grb.tostring())
            
            mslp = gaussian_filter(grb['values'], sigma=3.0) * units.pascal
        
        # Soil moisture
        elif grb.indicatorOfParameter == 144:
    
            # Modify soil moisture values
            grb['values'] = np.ma.array(grb.values + (ds_soil['Soil Moisture'].sel(depth=grb.level).values - 1), mask=False)
            
            grbout.write(grb.tostring())
    
        else:
            # If the message is not any of the desired variables, continue onto the next one
            None
    
    # Close the grib file
    grbout.close()
    
    # Create a plot of 500-hPa Height and Wind
    
    if setting == 'Control':
        model=''
    
    if plot_3d == True:
    
        # Calculate wind speed from components
        sped_500 = mpcalc.wind_speed(uwnd_500, vwnd_500).to('kt')
    
        # Set up the projection that will be used for plotting
        mapcrs = ccrs.LambertConformal(central_longitude=-100,
                                       central_latitude=35,
                                       standard_parallels=(30, 60))
    
        # Set up the projection of the data; if lat/lon then PlateCarree is what you want
        datacrs = ccrs.PlateCarree()
    
        # 500 
        # Start the figure and create plot axes with proper projection
        fig = plt.figure(1, figsize=(10, 8))
        ax = plt.subplot(111, projection=mapcrs)
        ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    
        # Add geopolitical boundaries for map reference
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
        ax.add_feature(cfeature.STATES.with_scale('50m'))
    
        # Plot 500-hPa Colorfill Wind Speeds in knots
        clevs_500_sped = np.arange(30, 100, 10)
        cf = ax.contourf(lons, lats, sped_500, clevs_500_sped, cmap=plt.cm.BuPu,
                         transform=datacrs)
        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
        cb.set_label('Wind Speed (kt)')
    
        #Plot 500-hPa Geopotential Heights in meters
        #clevs_500_hght = np.arange(0, 8000, 60)
        #cs = ax.contour(lons, lats, hght_500, clevs_500_hght, colors='black',
        #                transform=datacrs)
        #plt.clabel(cs, fmt='%d')
    
        # Plot 500-hPa wind barbs in knots, regrid to reduce number of barbs
    
        # Make some nice titles for the plot (one right, one left)
        plt.title('500-hPa NAM Wind Speed (kt), '+setting+' '+model, fontsize=8, loc='left')
        plt.title(dt.strftime('%m-%d-%Y %H:%M UTC'), loc='right')
    
        # Adjust image and show
        plt.subplots_adjust(bottom=0, top=1)
        plt.tight_layout()
        plt.savefig('../figures/Atmos/500_Winds_'+setting+'_'+str(hour)+'UTC'+'_'+file[11:-8]+'_'+model+'.png', dpi=200)
        plt.close()
    
        # Begin figure and set CONUS areal extent
        fig = plt.figure(1, figsize=(10, 8))
        ax = plt.subplot(111, projection=mapcrs)
        ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())

        # Add coastlines and state boundaries
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
        ax.add_feature(cfeature.STATES.with_scale('50m'))

        # Plot Colorfill of 700-hPa relative humidity -
        # normalize color to not have too dark of green at the top end
        specific_hum_700 = np.linspace(relh_700.min(), relh_700.max(), 10)
        cf = ax.contourf(lons, lats, relh_700, specific_hum_700, cmap=plt.cm.Greens,
                         transform=datacrs)
        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
        cb.set_label('Specific Humidity (g/kg)')

        ## Plot contours of 700-hPa geopotential height in black, solid lines
        #clevs_700_hght = np.arange(0, 8000, 30)
        #cs2 = ax.contour(lons, lats, hght_700, clevs_700_hght, colors='black', transform=datacrs)
        #ax.clabel(cs2, fmt='%d', fontsize='large')

        # Add some useful titles
        plt.title('700-hPa NAM Rel. Hum (%), '+setting+' '+ str(hour) + ' UTC '+model, fontsize=8, loc='left')
        plt.title(dt.strftime('%m-%d-%Y %H:%M UTC'), loc='right')

        plt.savefig('../figures/Atmos/Specific_hum_'+setting+'_'+str(hour)+'_UTC'+'_'+file[11:-8]+'_'+model+'.png', dpi=200)
        plt.close()
    
    if plot_sfc == True:
    
        # Set up the projection that will be used for plotting
        mapcrs = ccrs.LambertConformal(central_longitude=-100,
                                       central_latitude=35,
                                       standard_parallels=(30, 60))
    
        # Set up the projection of the data; if lat/lon then PlateCarree is what you want
        datacrs = ccrs.PlateCarree()
    
        # 500 
        # Start the figure and create plot axes with proper projection
        fig = plt.figure(1, figsize=(10, 8))
        ax = plt.subplot(111, projection=mapcrs)
        ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    
        # Add geopolitical boundaries for map reference
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
        ax.add_feature(cfeature.STATES.with_scale('50m'))
    
        # Plot surface temperature in degrees Farenheit
        clevs_temp = np.arange(10, 120, 5)
        cf = ax.contourf(lons, lats, tmpk.to('degF'), clevs_temp, cmap=plt.cm.gist_rainbow_r,
                         transform=datacrs)
        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
        cb.set_label('Temperature (F)')
    
        #Plot MSLP in hPa
        # Plot MSLP
        clevmslp = np.arange(950, 1030., 4)
        cs2 = ax.contour(lons, lats, mslp.to('hPa'), clevmslp, colors='k', linewidths=1.25,
                         linestyles='solid', transform=datacrs)
        ax.clabel(cs2, fmt='%d')
    
        # Plot 500-hPa wind barbs in knots, regrid to reduce number of barbs
    
        # Make some nice titles for the plot (one right, one left)
        plt.title('Surface Pressure (hPa), Surface Temperature (degF) '+setting+' '+str(hour)+' UTC '+model, fontsize=8, loc='left')
        plt.title(dt.strftime('%m-%d-%Y %H:%M UTC'), fontsize=8, loc='right')
    
        # Adjust image and show
        plt.subplots_adjust(bottom=0, top=1)
        plt.tight_layout()
        plt.savefig('../figures/Surface/Sfc_Pres_Temp_'+setting+'_'+str(hour)+'_UTC'+'_'+file[11:-8]+'_'+model+'_3hour.png', dpi=200)
        plt.close()
        
    if plot_soil == True:
        
        # Set up the projection that will be used for plotting
        mapcrs = ccrs.LambertConformal(central_longitude=-100,
                                       central_latitude=35,
                                       standard_parallels=(30, 60))
    
        # Set up the projection of the data; if lat/lon then PlateCarree is what you want
        datacrs = ccrs.PlateCarree()
    
        # 500 
        # Start the figure and create plot axes with proper projection
        fig = plt.figure(1, figsize=(10, 8))
        ax = plt.subplot(111, projection=mapcrs)
        ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    
        # Add geopolitical boundaries for map reference
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
        ax.add_feature(cfeature.STATES.with_scale('50m'))
    
        # Plot surface temperature in degrees Farenheit
        clevs_temp = np.arange(10, 120, 5)
        cf = ax.contourf(lons, lats, soil_t.to('degF'), clevs_temp, cmap=plt.cm.gist_rainbow_r,
                         transform=datacrs)

        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
        cb.set_label('Temperature (F)')
    
        # Plot 500-hPa wind barbs in knots, regrid to reduce number of barbs
    
        # Make some nice titles for the plot (one right, one left)
        plt.title('Soil Temperature (F) '+setting+' '+model, loc='left')
        plt.title(dt.strftime('%m-%d-%Y %H:%M UTC'), loc='right')
    
        # Adjust image and show
        plt.subplots_adjust(bottom=0, top=1)
        plt.tight_layout()
        plt.savefig('../figures/Soil/Soil_Temp_'+setting+'_'+file[11:-8]+'_'+model+'3_hour.png', dpi=200)
        plt.close()
    
    
    # Provide an update on the progress
    print('Finish Writing ', model)

### Loop through the 3 models that were used in this study
- MIROC
- GFDL
- NCAR CCSM

In [4]:
def mod_files(setting, time_frequency='hourly', plot_3d=False, plot_sfc=False, plot_soil=False,
              nam_filepath='/data/keeling/a/mjwoods3/data/moore/NAM_forecast/',
              new_filepath='/data/keeling/a/mgrover4/b/CMIP5_to_WRF/modified_files/',
              models = ['MIROC', 'GFDL', 'NCAR']):
    
    new_filepath = new_filepath + time_frequency
    files = os.listdir(nam_filepath)
    for file in files:
        if 'nam' in file:
            # Loop through the different models
            for model in models:
    
                # Use the function defined in the previous cell
                modify_grib_file(model, nam_filepath, file, new_filepath, setting, time_frequency, plot_3d, plot_sfc, plot_soil)
        
        print(file)

In [5]:
import metpy.calc as mcalc
import warnings

import warnings
warnings.filterwarnings('ignore')

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

### Calculate the hourly files

In [9]:
mod_files('Control')

Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last)

hourly


Found latitude/longitude values, assuming latitude_longitude for projection grid_mapping variable
Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/dat

Missing variable:  ts
Using TAS for TS
362:Snow depth:m (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305201200
379:223:223 (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305201200
Finish Writing  
hourly


Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last)

...

Missing variable:  ts
Using TAS for TS
362:Snow depth:m (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305210600
379:223:223 (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305210600
Finish Writing  
hourly


Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130521_0600_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130521_0600_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130521_0600_000.grb.011ab.idx'
Traceback (most recent call last)

Missing variable:  ts
Using TAS for TS
362:Snow depth:m (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305210600
379:223:223 (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305210600
Finish Writing  
hourly


Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130521_0600_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130521_0600_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130521_0600_000.grb.011ab.idx'
Traceback (most recent call last)

362:Snow depth:m (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305210600
379:223:223 (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305210600
Finish Writing  
namanl_218_20130521_0600_000.grb


In [10]:
mod_files('Atmos')

Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last)

hourly


Found latitude/longitude values, assuming latitude_longitude for projection grid_mapping variable
Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/dat

Missing variable:  ts
Using TAS for TS
362:Snow depth:m (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305201200
379:223:223 (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305201200
Finish Writing  MIROC
hourly


Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last)

...

Missing variable:  ts
Using TAS for TS
362:Snow depth:m (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305210600
379:223:223 (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305210600


Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130521_0600_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130521_0600_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130521_0600_000.grb.011ab.idx'
Traceback (most recent call last)

Finish Writing  GFDL
hourly


Found latitude/longitude values, assuming latitude_longitude for projection grid_mapping variable
Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130521_0600_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130521_0600_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/dat

362:Snow depth:m (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305210600
379:223:223 (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305210600
Finish Writing  NCAR
namanl_218_20130521_0600_000.grb


In [ ]:
mod_files('Surface')

Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last)

hourly


Found latitude/longitude values, assuming latitude_longitude for projection grid_mapping variable
Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/dat

Using TAS for TS
362:Snow depth:m (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305201200
379:223:223 (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305201200


Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1200_000.grb.011ab.idx'
Traceback (most recent call last)

...

Found latitude/longitude values, assuming latitude_longitude for projection grid_mapping variable
Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1800_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1800_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/dat

Using TAS for TS
362:Snow depth:m (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305201800
379:223:223 (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305201800
Finish Writing  MIROC
hourly


Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1800_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1800_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1800_000.grb.011ab.idx'
Traceback (most recent call last)

Using TAS for TS
362:Snow depth:m (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305201800
379:223:223 (instant):lambert:surface:level 0:fcst time 0 hrs:from 201305201800
Finish Writing  GFDL
hourly


Can't create file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1800_000.grb.011ab.idx'
Traceback (most recent call last):
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 305, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/data/keeling/a/mgrover4/a/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/cfgrib/messages.py", line 242, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1800_000.grb.011ab.idx'
Can't read index file '/data/keeling/a/mjwoods3/data/moore/NAM_forecast/namanl_218_20130520_1800_000.grb.011ab.idx'
Traceback (most recent call last)

In [ ]:
mod_files('Soil')

In [ ]:
mod_files('All')

### Calcualte the monthly files

In [ ]:
mod_files('Control', 'monthly')

In [ ]:
mod_files('Atmos', 'monthly')

In [ ]:
mod_files('Surface', 'monthly')

In [ ]:
mod_files('Soil', 'monthly')

In [ ]:
mod_files('All', 'monthly')